# Interactive Chart with Cross-Highlight
This example is a modification of the [Interactive Chart with Cross-Highlight](https://altair-viz.github.io/gallery/interactive_cross_highlight.html) Altair gallery example.  The dataset size has been increased from 3.2k rows to 200k rows.  At this data size, the plain Altair version has noticeable lag while the VegaFusion version is still responsive. A VegaFusion version with 1 million rows is also included.

In [ ]:
import altair as alt
from vega_datasets import data
import pandas as pd
import vegafusion as vf

In [ ]:
def make_chart(data_source):
    pts = alt.selection(type="multi", encodings=['x'])

    rect = alt.Chart(data_source).mark_rect().encode(
        alt.X('IMDB_Rating:Q', bin=True),
        alt.Y('Rotten_Tomatoes_Rating:Q', bin=True),
        alt.Color('count()',
            scale=alt.Scale(scheme='greenblue'),
            legend=alt.Legend(title='Total Records')
        )
    )

    circ = rect.mark_point().encode(
        alt.ColorValue('grey'),
        alt.Size('count()',
            legend=alt.Legend(title='Records in Selection')
        )
    ).transform_filter(
        pts
    ).properties(
        width=300,
        height=250
    )

    bar = alt.Chart(data_source).mark_bar().encode(
        x='Major_Genre:N',
        y='count()',
        color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
    ).properties(
        width=300,
        height=250
    ).add_selection(pts)

    return alt.hconcat(
        bar,
        rect + circ,
    ).resolve_legend(
        color="independent",
        size="independent"
    )

## Plain Altair
#### 201k rows

In [ ]:
vf.jupyter.disable()
source = "https://vegafusion-datasets.s3.amazonaws.com/vega/movies_201k.json"
make_chart(source)

## Altair with VegaFusion
#### 201k rows

In [ ]:
vf.jupyter.enable(download_source_link="https://github.com/vegafusion/demos")
source_data = pd.read_parquet(
    "https://vegafusion-datasets.s3.amazonaws.com/vega/movies_201k.parquet"
)
make_chart(source_data)

In [ ]:
# ## Altair with VegaFusion
# #### 1 million rows

# source_data = pd.read_parquet(
#     "https://vegafusion-datasets.s3.amazonaws.com/vega/movies_1m.parquet"
# )
# make_chart(source_data)